**Unit Commitment Seminar — ANSWER KEY**

**Power Markets and Regulations**

**TA: Sahar Moghimian**

# Unit Commitment

_**[Power Systems Optimization](https://github.com/east-winds/power-systems-optimization)**_

_by Michael R. Davidson and Jesse D. Jenkins (last updated: October 11, 2022)_

This notebook will build on the economic dispatch (ED) model by introducing binary startup (called "commitment") decisions, constraints, and costs of thermal generators. Similar to ED, it minimizes the short-run production costs of meeting electricity demand, but these additions are necessary for operating systems with large amounts of (inflexible) thermal generation. They also entail significant computational trade-offs.

****

We build up the model in several stages. We first start with the simplest model that incorporates unit commitment decisions in order to understand the logic of three-variable commitment formulations.

Next, we introduce moderate complexity through the addition of ramp constraints&mdash;which we need to be modified from the ED to account for startups and shutdowns.

Finally, we model a more realistic unit commitment that includes reserves&mdash;an important feature of day-ahead scheduling in power systems.

## Introduction to unit commitment

Engineering considerations severely limit the possible output ranges of power plants. System operators need to be aware of these limitations when scheduling generation to meet demand. Thermal power plants, in particular, due to their complex designs, thermodynamic cycles, and material properties, can be particularly challenging. In practice, due to the times involved in bringing these power plants online, much of this scheduling is done day-ahead, which gives rise to the need for a day-ahead market.

**Unit commitment** (UC) is the problem of minimizing short-run costs of production inclusive of production and startup costs, in order to meet a given demand and considering relevant engineering constraints of the generators. It is built on the ED model in that it typically considers all of the same variables in addition to the new startup-related variables and constraints&mdash;hence, it is also sometimes called unit commitment and economic dispatch (UCED). This notebook, for simplicity, only considers a single-bus case, while typical formulations usually include a simplified network representation.

## Simple unit commitment

We start with the simplest case that incorporates unit commitment, building on the `economic_dispatch_multi` problem of the [Economic Dispatch notebook](04-Economic-Dispatch.ipynb).


$$
\begin{align}
\min \ & \sum_{g \in G, t \in T} VarCost_g \times GEN_{g,t} + \sum_{g \in G_{thermal}, t \in T} StartUpCost_g \times START_{g,t} \\
\end{align}
$$
$$
\begin{align}
\text{s.t.} & \\
 & \sum_{g} GEN_{g,t} = Demand_t & \forall \quad t \in T \\
 & GEN_{g,t} \leq Pmax_{g,t} & \forall \quad g \notin G_{thermal} , t \in T \\
 & GEN_{g,t} \geq Pmin_{g,t} & \forall \quad g \notin G_{thermal} , t \in T \\
 & GEN_{g,t} \leq Pmax_{g,t} \times COMMIT_{g,t} & \forall \quad g \in G_{thermal} , t \in T \\
 & GEN_{g,t} \geq Pmin_{g,t} \times COMMIT_{g,t} & \forall \quad g \in G_{thermal} , t \in T \\
 & COMMIT_{g,t} \geq \sum_{t'\geq t-MinUp_g}^{t} START_{g,t} & \forall \quad g \in G_{thermal} , t \in T \\
 & 1-COMMIT_{g,t} \geq \sum_{t'\geq t-MinDown_g}^{t} SHUT_{g,t} &\forall \quad g \in G_{thermal} , t \in T \\
  & COMMIT_{g,t+1} - COMMIT_{g,t} =&\\
 & \quad START_{g,t+1} - SHUT_{g,t+1} &\forall \quad G_{thermal} \in G , t = 1..T-1
\end{align}
$$


The **decision variables** in the above problem:

- $GEN_{g}$, generation (in MW) produced by each generator, $g$
- $START_{g,t}$, startup decision (binary) of thermal generator $g$ at time $t$
- $SHUT_{g,t}$, shutdown decision (binary) of thermal generator $g$ at time $t$
- $COMMIT_{g,t}$, commitment status (binary) of generator $g$ at time $t$

The **parameters** are:

- $Pmin_g$, the minimum operating bounds for generator $g$ (based on engineering or natural resource constraints)
- $Pmax_g$, the maximum operating bounds for generator $g$ (based on engineering or natural resource constraints)
- $Demand$, the demand (in MW)
- $VarCost_g = VarOM_g + HeatRate_g \times FuelCost_g$, the variable cost of generator $g$
- $StartUpCost_g$, the startup cost of generator $g$
- $MinUp_g$, the minimum up time of generator $g$, or the minimum time after start-up before a unit can shut down
- $MinDown_g$, the minimum down time of generator $g$, or the minimum time after shut-down before a unit can start again

In addition, we introduce a few different sets:

- $G$, the set of all generators
- $G_{thermal} \subset G$, the subset of thermal generators for which commitment is necessary
- $T$, the set of all time periods over which we are optimizing commitment and dispatch decisions

Finally, the **three-variable commitment equations** capture the basic logic of commitment:

- Units incur costs when they startup (not when they shutdown)
- Units must stay on (and off) for a minimum period of time&mdash;in lieu of explicitly enforcing a startup trajectory
- Some summations (simplified here) will need to be modified near the beginning of the time period

There are some further resources at the bottom for alternative and/or more complex formulations of the UC.

Now, let's implement UC.

### 0. Install and import packages

In [ ]:
# Install packages (uncomment if running in Google Colab or fresh environment)
# !pip install pyomo highspy plotly pandas

In [ ]:
import pyomo.environ as pyo
import pandas as pd
import numpy as np
import plotly.express as px

pd.set_option('display.max_columns', 120)
pd.set_option('display.max_rows', 100)

### 1. Load and format data

We will use data loosely based on San Diego Gas and Electric (SDG&E, via the [PowerGenome](https://github.com/gschivley/PowerGenome) data platform) including a few neighboring generators and adjustments to make the problem easy to solve, consisting of:

- 33 generators (we added a few more to ensure we can provide enough reserves)
- estimated hourly demand for 2020 (net load at the transmission substation level after subtracting 600MW of behind-the-meter solar from original demand)
- variable generation capacity factors
- estimated natural gas fuel costs

In order to demonstrate the impacts of unit commitment, we will keep our high solar sensitivity case from the [Economic Dispatch notebook](04-Economic-Dispatch.ipynb) (with 3,500 MW of solar PV) to produce large variations in net load (demand less available renewable supply) required to meet demand (similar to [California's infamous "Duck Curve"](https://www.caiso.com/Documents/FlexibleResourcesHelpRenewables_FastFacts.pdf)).

In [ ]:
# Load data from GitHub
gen_info = pd.read_csv("https://raw.githubusercontent.com/Power-Systems-Optimization-Course/power-systems-optimization/master/Notebooks/uc_data/Generators_data.csv")
fuels = pd.read_csv("https://raw.githubusercontent.com/Power-Systems-Optimization-Course/power-systems-optimization/master/Notebooks/uc_data/Fuels_data.csv")
loads = pd.read_csv("https://raw.githubusercontent.com/Power-Systems-Optimization-Course/power-systems-optimization/master/Notebooks/uc_data/Demand.csv")
gen_variable = pd.read_csv("https://raw.githubusercontent.com/Power-Systems-Optimization-Course/power-systems-optimization/master/Notebooks/uc_data/Generators_variability.csv")

# Rename all columns to lowercase
for df in [gen_info, fuels, loads, gen_variable]:
    df.columns = [c.lower() for c in df.columns]

**Construct generator dataframe**

In [ ]:
# Keep columns relevant to our UC model (columns 0-25, i.e. first 26 columns)
gen_info = gen_info.iloc[:, :26]

# Outer join with fuel costs
gen_df = gen_info.merge(fuels, on='fuel', how='outer')
gen_df.rename(columns={'cost_per_mmbtu': 'fuel_cost'}, inplace=True)
gen_df['fuel_cost'] = gen_df['fuel_cost'].fillna(0)

# Create "is_variable" column
gen_df['is_variable'] = gen_df['resource'].isin(
    ['onshore_wind_turbine', 'small_hydroelectric', 'solar_photovoltaic']
)

# Create full name of generator (including geographic location and cluster number)
# Note: cluster is cast to float then string to match the column names in the variability CSV
gen_df['gen_full'] = (gen_df['region'] + '_' + gen_df['resource'] + '_' + gen_df['cluster'].astype(float).astype(str)).str.lower()

# Remove generators with no capacity
gen_df = gen_df[gen_df['existing_cap_mw'] > 0].reset_index(drop=True)

gen_df.head()

**Modify load and variable generation dataframes**

In [ ]:
# 1. Convert from GMT to GMT-8
gen_variable['hour'] = (gen_variable['hour'] - 9) % 8760 + 1
gen_variable = gen_variable.sort_values('hour').reset_index(drop=True)
loads['hour'] = (loads['hour'] - 9) % 8760 + 1
loads = loads.sort_values('hour').reset_index(drop=True)

# 2. Convert from "wide" to "long" format using pd.melt
gen_variable_long = pd.melt(
    gen_variable,
    id_vars=['hour'],
    var_name='gen_full',
    value_name='cf'
)

gen_variable_long.head()

### 2. Create solver functions

As a utility function, since we do this a lot, we'll create a function to convert Pyomo variable outputs with two indexes to DataFrames.

In [ ]:
def value_to_df_2dim(var, index1, index2, val_col='gen'):
    """
    Convert a Pyomo variable indexed by (index1, index2) to a pandas DataFrame.
    
    Parameters:
        var      -- Pyomo Var object indexed by two sets
        index1   -- list of first-dimension index values (e.g., generator r_ids)
        index2   -- list of second-dimension index values (e.g., hours)
        val_col  -- name for the value column (default: 'gen')
    
    Returns:
        DataFrame with columns [r_id, hour, val_col]
    """
    records = []
    for i in index1:
        for t in index2:
            records.append({
                'r_id': i,
                'hour': int(t),
                val_col: pyo.value(var[i, t])
            })
    return pd.DataFrame(records)

Then we'll create a function to create and solve a simple unit commitment problem with specified input data.

In [ ]:
def unit_commitment_simple(gen_df, loads, gen_variable, mip_gap):
    """
    Solve simple unit commitment problem (commitment equations).
    """
    model = pyo.ConcreteModel('UC_Simple')

    G_thermal = list(gen_df.loc[gen_df['up_time'] > 0, 'r_id'])
    G_nonthermal = list(gen_df.loc[gen_df['up_time'] == 0, 'r_id'])
    G_var = list(gen_df.loc[gen_df['is_variable'] == True, 'r_id'])
    G_nonvar = list(gen_df.loc[gen_df['is_variable'] == False, 'r_id'])
    G_nt_nonvar = list(set(G_nonvar) & set(G_nonthermal))
    G = list(gen_df['r_id'])
    T = list(loads['hour'])
    T_red = T[:-1]

    gen_var_cf = gen_variable.merge(
        gen_df.loc[gen_df['is_variable'] == True, ['r_id', 'gen_full', 'existing_cap_mw']],
        on='gen_full', how='inner'
    )

    gen_lookup = gen_df.set_index('r_id')

    model.G = pyo.Set(initialize=G)
    model.G_thermal = pyo.Set(initialize=G_thermal)
    model.T = pyo.Set(initialize=T)

    model.GEN = pyo.Var(G, T, domain=pyo.NonNegativeReals)
    model.COMMIT = pyo.Var(G_thermal, T, domain=pyo.Binary)
    model.START = pyo.Var(G_thermal, T, domain=pyo.Binary)
    model.SHUT = pyo.Var(G_thermal, T, domain=pyo.Binary)

    def obj_rule(m):
        cost_nonvar = sum(
            (gen_lookup.at[i, 'heat_rate_mmbtu_per_mwh'] * gen_lookup.at[i, 'fuel_cost'] +
             gen_lookup.at[i, 'var_om_cost_per_mwh']) * m.GEN[i, t]
            for i in G_nonvar for t in T
        )
        cost_var = sum(
            gen_lookup.at[i, 'var_om_cost_per_mwh'] * m.GEN[i, t]
            for i in G_var for t in T
        )
        cost_start = sum(
            gen_lookup.at[i, 'start_cost_per_mw'] *
            gen_lookup.at[i, 'existing_cap_mw'] *
            m.START[i, t]
            for i in G_thermal for t in T
        )
        return cost_nonvar + cost_var + cost_start

    model.objective = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

    demand_dict = dict(zip(loads['hour'], loads['demand']))

    def demand_rule(m, t):
        return sum(m.GEN[i, t] for i in G) == demand_dict[t]
    model.cDemand = pyo.Constraint(T, rule=demand_rule)

    def cap_thermal_min_rule(m, i, t):
        return m.GEN[i, t] >= (m.COMMIT[i, t] *
                               gen_lookup.at[i, 'existing_cap_mw'] *
                               gen_lookup.at[i, 'min_power'])
    model.Cap_thermal_min = pyo.Constraint(G_thermal, T, rule=cap_thermal_min_rule)

    def cap_thermal_max_rule(m, i, t):
        return m.GEN[i, t] <= m.COMMIT[i, t] * gen_lookup.at[i, 'existing_cap_mw']
    model.Cap_thermal_max = pyo.Constraint(G_thermal, T, rule=cap_thermal_max_rule)

    def cap_nt_nonvar_rule(m, i, t):
        return m.GEN[i, t] <= gen_lookup.at[i, 'existing_cap_mw']
    model.Cap_nt_nonvar = pyo.Constraint(G_nt_nonvar, T, rule=cap_nt_nonvar_rule)

    model.Cap_var = pyo.ConstraintList()
    for _, row in gen_var_cf.iterrows():
        model.Cap_var.add(
            model.GEN[row['r_id'], row['hour']] <= row['cf'] * row['existing_cap_mw']
        )

    T_set = set(T)

    def startup_rule(m, i, t):
        up_time = int(gen_lookup.at[i, 'up_time'])
        relevant_t = [tt for tt in range(t - up_time, t + 1) if tt in T_set]
        return m.COMMIT[i, t] >= sum(m.START[i, tt] for tt in relevant_t)
    model.Startup = pyo.Constraint(G_thermal, T, rule=startup_rule)

    def shutdown_rule(m, i, t):
        down_time = int(gen_lookup.at[i, 'down_time'])
        relevant_t = [tt for tt in range(t - down_time, t + 1) if tt in T_set]
        return 1 - m.COMMIT[i, t] >= sum(m.SHUT[i, tt] for tt in relevant_t)
    model.Shutdown = pyo.Constraint(G_thermal, T, rule=shutdown_rule)

    def commitment_status_rule(m, i, t):
        t_next = t + 1
        return m.COMMIT[i, t_next] - m.COMMIT[i, t] == m.START[i, t_next] - m.SHUT[i, t_next]
    model.CommitmentStatus = pyo.Constraint(G_thermal, T_red, rule=commitment_status_rule)

    solver = pyo.SolverFactory('appsi_highs')
    solver.options['mip_rel_gap'] = mip_gap
    results = solver.solve(model, tee=True)

    gen = value_to_df_2dim(model.GEN, G, T, 'gen')
    commit = value_to_df_2dim(model.COMMIT, G_thermal, T, 'gen')

    curtail = gen_var_cf.merge(gen, on=['r_id', 'hour'], how='inner')
    curtail['curt'] = curtail['cf'] * curtail['existing_cap_mw'] - curtail['gen']

    cost = pyo.value(model.objective)
    status = results.solver.termination_condition

    return gen, commit, curtail, cost, status

### 3. Solve a day's unit commitment

In [ ]:
# A spring day
n = 100
T_period = list(range(n * 24 + 1, (n + 1) * 24 + 1))  # hours 2401-2424

# High solar case: 3,500 MW
gen_df_sens = gen_df.copy()
gen_df_sens.loc[gen_df_sens['resource'] == 'solar_photovoltaic', 'existing_cap_mw'] = 3500

loads_multi = loads[loads['hour'].isin(T_period)].reset_index(drop=True)
gen_variable_multi = gen_variable_long[gen_variable_long['hour'].isin(T_period)].reset_index(drop=True)

print(f"Time period: hours {T_period[0]} to {T_period[-1]}")
print(f"Number of generators: {len(gen_df_sens)}")
print(f"Number of hours: {len(T_period)}")

In [ ]:
sol_gen_simple, sol_commit_simple, sol_curtail_simple, cost_simple, status_simple = \
    unit_commitment_simple(gen_df_sens, loads_multi, gen_variable_multi, 0.01)

print(f"\nStatus: {status_simple}")
print(f"Total cost: ${cost_simple:,.2f}")

In [ ]:
def plot_generation(sol_gen_raw, sol_curtail, gen_df, gen_variable_multi, T_period, title='Generation Dispatch'):
    sol_gen = sol_gen_raw.merge(gen_df[['r_id', 'resource']], on='r_id', how='inner')
    sol_gen = sol_gen.groupby(['resource', 'hour'], as_index=False)['gen'].sum()

    rename_map = {
        'solar_photovoltaic': '_solar_photovoltaic',
        'onshore_wind_turbine': '_onshore_wind_turbine',
        'small_hydroelectric': '_small_hydroelectric'
    }
    sol_gen['resource'] = sol_gen['resource'].replace(rename_map)

    solar_cf = gen_variable_multi[
        gen_variable_multi['gen_full'] == 'wec_sdge_solar_photovoltaic_1.0'
    ][['hour', 'cf']].copy()
    btm = pd.DataFrame({
        'resource': '_solar_photovoltaic_btm',
        'hour': solar_cf['hour'].values,
        'gen': solar_cf['cf'].values * 600
    })
    sol_gen = pd.concat([sol_gen, btm], ignore_index=True)

    curtail = sol_curtail.groupby('hour', as_index=False)['curt'].sum()
    curtail['resource'] = '_curtailment'
    curtail.rename(columns={'curt': 'gen'}, inplace=True)
    sol_gen = pd.concat([sol_gen, curtail[['resource', 'hour', 'gen']]], ignore_index=True)

    sol_gen['hour'] = sol_gen['hour'] - T_period[0]
    sol_gen = sol_gen.sort_values(['resource', 'hour']).reset_index(drop=True)

    resources = sorted(sol_gen['resource'].unique())
    category10 = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
                  '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
    color_map = {r: category10[i % len(category10)] for i, r in enumerate(resources)}

    fig = px.area(
        sol_gen, x='hour', y='gen', color='resource',
        title=title,
        labels={'hour': 'Hour', 'gen': 'Generation (MW)', 'resource': 'Resource'},
        color_discrete_map=color_map
    )
    fig.update_layout(xaxis_title='Hour', yaxis_title='Generation (MW)')
    fig.show()

    return sol_gen

sol_gen_plot = plot_generation(
    sol_gen_simple, sol_curtail_simple, gen_df, gen_variable_multi, T_period,
    title='Simple UC - Generation Dispatch (Spring Day, 3500 MW Solar)'
)

In [ ]:
def plot_commitment(sol_commit_raw, gen_df, T_period, title='Commitment Status'):
    sol_commit = sol_commit_raw.merge(gen_df[['r_id', 'resource']], on='r_id', how='inner')
    sol_commit = sol_commit.groupby(['resource', 'hour'], as_index=False)['gen'].sum()
    sol_commit['hour'] = sol_commit['hour'] - T_period[0]

    resources = sorted(sol_commit['resource'].unique())
    category10 = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
                  '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
    color_map = {r: category10[i % len(category10)] for i, r in enumerate(resources)}

    fig = px.area(
        sol_commit, x='hour', y='gen', color='resource',
        title=title,
        labels={'hour': 'Hour', 'gen': 'Committed Units', 'resource': 'Resource'},
        color_discrete_map=color_map
    )
    fig.update_layout(xaxis_title='Hour', yaxis_title='Committed Units')
    fig.show()

plot_commitment(
    sol_commit_simple, gen_df, T_period,
    title='Simple UC - Commitment Status (Spring Day, 3500 MW Solar)'
)

## Moderate complexity unit commitment

We expand the above unit commitment with ramp equations.

In [ ]:
def unit_commitment_ramp(gen_df, loads, gen_variable, mip_gap):
    """
    Solve moderate complexity unit commitment problem (commitment and ramp equations).
    """
    model = pyo.ConcreteModel('UC_Ramp')

    G_thermal = list(gen_df.loc[gen_df['up_time'] > 0, 'r_id'])
    G_nonthermal = list(gen_df.loc[gen_df['up_time'] == 0, 'r_id'])
    G_var = list(gen_df.loc[gen_df['is_variable'] == True, 'r_id'])
    G_nonvar = list(gen_df.loc[gen_df['is_variable'] == False, 'r_id'])
    G_nt_nonvar = list(set(G_nonvar) & set(G_nonthermal))
    G = list(gen_df['r_id'])
    T = list(loads['hour'])
    T_red = T[:-1]

    gen_var_cf = gen_variable.merge(
        gen_df.loc[gen_df['is_variable'] == True, ['r_id', 'gen_full', 'existing_cap_mw']],
        on='gen_full', how='inner'
    )

    gen_lookup = gen_df.set_index('r_id')

    model.G = pyo.Set(initialize=G)
    model.G_thermal = pyo.Set(initialize=G_thermal)
    model.T = pyo.Set(initialize=T)

    model.GEN = pyo.Var(G, T, domain=pyo.NonNegativeReals)
    model.GENAUX = pyo.Var(G_thermal, T, domain=pyo.NonNegativeReals)
    model.COMMIT = pyo.Var(G_thermal, T, domain=pyo.Binary)
    model.START = pyo.Var(G_thermal, T, domain=pyo.Binary)
    model.SHUT = pyo.Var(G_thermal, T, domain=pyo.Binary)

    def obj_rule(m):
        cost_nonvar = sum(
            (gen_lookup.at[i, 'heat_rate_mmbtu_per_mwh'] * gen_lookup.at[i, 'fuel_cost'] +
             gen_lookup.at[i, 'var_om_cost_per_mwh']) * m.GEN[i, t]
            for i in G_nonvar for t in T
        )
        cost_var = sum(
            gen_lookup.at[i, 'var_om_cost_per_mwh'] * m.GEN[i, t]
            for i in G_var for t in T
        )
        cost_start = sum(
            gen_lookup.at[i, 'start_cost_per_mw'] *
            gen_lookup.at[i, 'existing_cap_mw'] *
            m.START[i, t]
            for i in G_thermal for t in T
        )
        return cost_nonvar + cost_var + cost_start

    model.objective = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

    demand_dict = dict(zip(loads['hour'], loads['demand']))

    def demand_rule(m, t):
        return sum(m.GEN[i, t] for i in G) == demand_dict[t]
    model.cDemand = pyo.Constraint(T, rule=demand_rule)

    def cap_thermal_min_rule(m, i, t):
        return m.GEN[i, t] >= (m.COMMIT[i, t] *
                               gen_lookup.at[i, 'existing_cap_mw'] *
                               gen_lookup.at[i, 'min_power'])
    model.Cap_thermal_min = pyo.Constraint(G_thermal, T, rule=cap_thermal_min_rule)

    def cap_thermal_max_rule(m, i, t):
        return m.GEN[i, t] <= m.COMMIT[i, t] * gen_lookup.at[i, 'existing_cap_mw']
    model.Cap_thermal_max = pyo.Constraint(G_thermal, T, rule=cap_thermal_max_rule)

    def cap_nt_nonvar_rule(m, i, t):
        return m.GEN[i, t] <= gen_lookup.at[i, 'existing_cap_mw']
    model.Cap_nt_nonvar = pyo.Constraint(G_nt_nonvar, T, rule=cap_nt_nonvar_rule)

    model.Cap_var = pyo.ConstraintList()
    for _, row in gen_var_cf.iterrows():
        model.Cap_var.add(
            model.GEN[row['r_id'], row['hour']] <= row['cf'] * row['existing_cap_mw']
        )

    T_set = set(T)

    def startup_rule(m, i, t):
        up_time = int(gen_lookup.at[i, 'up_time'])
        relevant_t = [tt for tt in range(t - up_time, t + 1) if tt in T_set]
        return m.COMMIT[i, t] >= sum(m.START[i, tt] for tt in relevant_t)
    model.Startup = pyo.Constraint(G_thermal, T, rule=startup_rule)

    def shutdown_rule(m, i, t):
        down_time = int(gen_lookup.at[i, 'down_time'])
        relevant_t = [tt for tt in range(t - down_time, t + 1) if tt in T_set]
        return 1 - m.COMMIT[i, t] >= sum(m.SHUT[i, tt] for tt in relevant_t)
    model.Shutdown = pyo.Constraint(G_thermal, T, rule=shutdown_rule)

    def commitment_status_rule(m, i, t):
        t_next = t + 1
        return m.COMMIT[i, t_next] - m.COMMIT[i, t] == m.START[i, t_next] - m.SHUT[i, t_next]
    model.CommitmentStatus = pyo.Constraint(G_thermal, T_red, rule=commitment_status_rule)

    def auxgen_rule(m, i, t):
        return m.GENAUX[i, t] == (m.GEN[i, t] -
                                  m.COMMIT[i, t] *
                                  gen_lookup.at[i, 'existing_cap_mw'] *
                                  gen_lookup.at[i, 'min_power'])
    model.AuxGen = pyo.Constraint(G_thermal, T, rule=auxgen_rule)

    def rampup_thermal_rule(m, i, t):
        t_next = t + 1
        return (m.GENAUX[i, t_next] - m.GENAUX[i, t] <=
                gen_lookup.at[i, 'existing_cap_mw'] *
                gen_lookup.at[i, 'ramp_up_percentage'])
    model.RampUp_thermal = pyo.Constraint(G_thermal, T_red, rule=rampup_thermal_rule)

    def rampdn_thermal_rule(m, i, t):
        t_next = t + 1
        return (m.GENAUX[i, t] - m.GENAUX[i, t_next] <=
                gen_lookup.at[i, 'existing_cap_mw'] *
                gen_lookup.at[i, 'ramp_dn_percentage'])
    model.RampDn_thermal = pyo.Constraint(G_thermal, T_red, rule=rampdn_thermal_rule)

    def rampup_nonthermal_rule(m, i, t):
        t_next = t + 1
        return (m.GEN[i, t_next] - m.GEN[i, t] <=
                gen_lookup.at[i, 'existing_cap_mw'] *
                gen_lookup.at[i, 'ramp_up_percentage'])
    model.RampUp_nonthermal = pyo.Constraint(G_nonthermal, T_red, rule=rampup_nonthermal_rule)

    def rampdn_rule(m, i, t):
        t_next = t + 1
        return (m.GEN[i, t] - m.GEN[i, t_next] <=
                gen_lookup.at[i, 'existing_cap_mw'] *
                gen_lookup.at[i, 'ramp_dn_percentage'])
    model.RampDn = pyo.Constraint(G, T_red, rule=rampdn_rule)

    solver = pyo.SolverFactory('appsi_highs')
    solver.options['mip_rel_gap'] = mip_gap
    results = solver.solve(model, tee=True)

    gen = value_to_df_2dim(model.GEN, G, T, 'gen')
    commit = value_to_df_2dim(model.COMMIT, G_thermal, T, 'gen')

    curtail = gen_var_cf.merge(gen, on=['r_id', 'hour'], how='inner')
    curtail['curt'] = curtail['cf'] * curtail['existing_cap_mw'] - curtail['gen']

    cost = pyo.value(model.objective)
    status = results.solver.termination_condition

    return gen, commit, curtail, cost, status

In [ ]:
sol_gen_ramp, sol_commit_ramp, sol_curtail_ramp, cost_ramp, status_ramp = \
    unit_commitment_ramp(gen_df_sens, loads_multi, gen_variable_multi, 0.01)

print(f"\nStatus: {status_ramp}")
print(f"Total cost: ${cost_ramp:,.2f}")

In [ ]:
sol_gen_ramp_plot = plot_generation(
    sol_gen_ramp, sol_curtail_ramp, gen_df, gen_variable_multi, T_period,
    title='UC with Ramp - Generation Dispatch (Spring Day, 3500 MW Solar)'
)

In [ ]:
plot_commitment(
    sol_commit_ramp, gen_df, T_period,
    title='UC with Ramp - Commitment Status (Spring Day, 3500 MW Solar)'
)

## Unit commitment with ramping and reserves

In [ ]:
def unit_commitment_full(gen_df, loads, gen_variable, mip_gap):
    """
    Solve full unit commitment problem (commitment, ramp, reserve equations).
    """
    model = pyo.ConcreteModel('UC_Full')

    G_thermal = list(gen_df.loc[gen_df['up_time'] > 0, 'r_id'])
    G_nonthermal = list(gen_df.loc[gen_df['up_time'] == 0, 'r_id'])
    G_var = list(gen_df.loc[gen_df['is_variable'] == True, 'r_id'])
    G_nonvar = list(gen_df.loc[gen_df['is_variable'] == False, 'r_id'])
    G_nt_nonvar = list(set(G_nonvar) & set(G_nonthermal))
    G = list(gen_df['r_id'])
    T = list(loads['hour'])
    T_red = T[:-1]

    gen_var_cf = gen_variable.merge(
        gen_df.loc[gen_df['is_variable'] == True, ['r_id', 'gen_full', 'existing_cap_mw']],
        on='gen_full', how='inner'
    )
    gen_var_cf['max_gen'] = gen_var_cf['cf'] * gen_var_cf['existing_cap_mw']

    gen_lookup = gen_df.set_index('r_id')

    demand_dict = dict(zip(loads['hour'], loads['demand']))
    ResReqUp = {t: 300 + 0.05 * demand_dict[t] for t in T}
    ResReqDn = {t: 0.05 * demand_dict[t] for t in T}

    model.G = pyo.Set(initialize=G)
    model.G_thermal = pyo.Set(initialize=G_thermal)
    model.T = pyo.Set(initialize=T)

    model.GEN = pyo.Var(G, T, domain=pyo.NonNegativeReals)
    model.GENAUX = pyo.Var(G_thermal, T, domain=pyo.NonNegativeReals)
    model.COMMIT = pyo.Var(G_thermal, T, domain=pyo.Binary)
    model.START = pyo.Var(G_thermal, T, domain=pyo.Binary)
    model.SHUT = pyo.Var(G_thermal, T, domain=pyo.Binary)
    model.RESUP = pyo.Var(G_thermal, T, domain=pyo.NonNegativeReals)
    model.RESDN = pyo.Var(G_thermal, T, domain=pyo.NonNegativeReals)

    def obj_rule(m):
        cost_nonvar = sum(
            (gen_lookup.at[i, 'heat_rate_mmbtu_per_mwh'] * gen_lookup.at[i, 'fuel_cost'] +
             gen_lookup.at[i, 'var_om_cost_per_mwh']) * m.GEN[i, t]
            for i in G_nonvar for t in T
        )
        cost_var = sum(
            gen_lookup.at[i, 'var_om_cost_per_mwh'] * m.GEN[i, t]
            for i in G_var for t in T
        )
        cost_start = sum(
            gen_lookup.at[i, 'start_cost_per_mw'] *
            gen_lookup.at[i, 'existing_cap_mw'] *
            m.START[i, t]
            for i in G_thermal for t in T
        )
        return cost_nonvar + cost_var + cost_start

    model.objective = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

    def demand_rule(m, t):
        return sum(m.GEN[i, t] for i in G) == demand_dict[t]
    model.cDemand = pyo.Constraint(T, rule=demand_rule)

    def cap_thermal_min_rule(m, i, t):
        return m.GEN[i, t] >= (m.COMMIT[i, t] *
                               gen_lookup.at[i, 'existing_cap_mw'] *
                               gen_lookup.at[i, 'min_power'])
    model.Cap_thermal_min = pyo.Constraint(G_thermal, T, rule=cap_thermal_min_rule)

    def cap_thermal_max_rule(m, i, t):
        return m.GEN[i, t] <= m.COMMIT[i, t] * gen_lookup.at[i, 'existing_cap_mw']
    model.Cap_thermal_max = pyo.Constraint(G_thermal, T, rule=cap_thermal_max_rule)

    def cap_nt_nonvar_rule(m, i, t):
        return m.GEN[i, t] <= gen_lookup.at[i, 'existing_cap_mw']
    model.Cap_nt_nonvar = pyo.Constraint(G_nt_nonvar, T, rule=cap_nt_nonvar_rule)

    model.Cap_var = pyo.ConstraintList()
    for _, row in gen_var_cf.iterrows():
        model.Cap_var.add(
            model.GEN[row['r_id'], row['hour']] <= row['max_gen']
        )

    T_set = set(T)

    def startup_rule(m, i, t):
        up_time = int(gen_lookup.at[i, 'up_time'])
        relevant_t = [tt for tt in range(t - up_time, t + 1) if tt in T_set]
        return m.COMMIT[i, t] >= sum(m.START[i, tt] for tt in relevant_t)
    model.Startup = pyo.Constraint(G_thermal, T, rule=startup_rule)

    def shutdown_rule(m, i, t):
        down_time = int(gen_lookup.at[i, 'down_time'])
        relevant_t = [tt for tt in range(t - down_time, t + 1) if tt in T_set]
        return 1 - m.COMMIT[i, t] >= sum(m.SHUT[i, tt] for tt in relevant_t)
    model.Shutdown = pyo.Constraint(G_thermal, T, rule=shutdown_rule)

    def commitment_status_rule(m, i, t):
        t_next = t + 1
        return m.COMMIT[i, t_next] - m.COMMIT[i, t] == m.START[i, t_next] - m.SHUT[i, t_next]
    model.CommitmentStatus = pyo.Constraint(G_thermal, T_red, rule=commitment_status_rule)

    def auxgen_rule(m, i, t):
        return m.GENAUX[i, t] == (m.GEN[i, t] -
                                  m.COMMIT[i, t] *
                                  gen_lookup.at[i, 'existing_cap_mw'] *
                                  gen_lookup.at[i, 'min_power'])
    model.AuxGen = pyo.Constraint(G_thermal, T, rule=auxgen_rule)

    def rampup_thermal_rule(m, i, t):
        t_next = t + 1
        return (m.GENAUX[i, t_next] - m.GENAUX[i, t] <=
                gen_lookup.at[i, 'existing_cap_mw'] *
                gen_lookup.at[i, 'ramp_up_percentage'])
    model.RampUp_thermal = pyo.Constraint(G_thermal, T_red, rule=rampup_thermal_rule)

    def rampdn_thermal_rule(m, i, t):
        t_next = t + 1
        return (m.GENAUX[i, t] - m.GENAUX[i, t_next] <=
                gen_lookup.at[i, 'existing_cap_mw'] *
                gen_lookup.at[i, 'ramp_dn_percentage'])
    model.RampDn_thermal = pyo.Constraint(G_thermal, T_red, rule=rampdn_thermal_rule)

    def rampup_nonthermal_rule(m, i, t):
        t_next = t + 1
        return (m.GEN[i, t_next] - m.GEN[i, t] <=
                gen_lookup.at[i, 'existing_cap_mw'] *
                gen_lookup.at[i, 'ramp_up_percentage'])
    model.RampUp_nonthermal = pyo.Constraint(G_nonthermal, T_red, rule=rampup_nonthermal_rule)

    def rampdn_rule(m, i, t):
        t_next = t + 1
        return (m.GEN[i, t] - m.GEN[i, t_next] <=
                gen_lookup.at[i, 'existing_cap_mw'] *
                gen_lookup.at[i, 'ramp_dn_percentage'])
    model.RampDn = pyo.Constraint(G, T_red, rule=rampdn_rule)

    def resupcap_rule(m, i, t):
        return m.RESUP[i, t] <= (m.COMMIT[i, t] * gen_lookup.at[i, 'existing_cap_mw']
                                 - m.GEN[i, t])
    model.ResUpCap = pyo.Constraint(G_thermal, T, rule=resupcap_rule)

    def resdncap_rule(m, i, t):
        return m.RESDN[i, t] <= (m.GEN[i, t] -
                                 m.COMMIT[i, t] *
                                 gen_lookup.at[i, 'existing_cap_mw'] *
                                 gen_lookup.at[i, 'min_power'])
    model.ResDnCap = pyo.Constraint(G_thermal, T, rule=resdncap_rule)

    def resupramp_rule(m, i, t):
        return m.RESUP[i, t] <= (gen_lookup.at[i, 'existing_cap_mw'] *
                                 gen_lookup.at[i, 'ramp_up_percentage'])
    model.ResUpRamp = pyo.Constraint(G_thermal, T, rule=resupramp_rule)

    def resdnramp_rule(m, i, t):
        return m.RESDN[i, t] <= (gen_lookup.at[i, 'existing_cap_mw'] *
                                 gen_lookup.at[i, 'ramp_dn_percentage'])
    model.ResDnRamp = pyo.Constraint(G_thermal, T, rule=resdnramp_rule)

    def resupreq_rule(m, t):
        return sum(m.RESUP[i, t] for i in G_thermal) >= ResReqUp[t]
    model.ResUpRequirement = pyo.Constraint(T, rule=resupreq_rule)

    def resdnreq_rule(m, t):
        return sum(m.RESDN[i, t] for i in G_thermal) >= ResReqDn[t]
    model.ResDnRequirement = pyo.Constraint(T, rule=resdnreq_rule)

    solver = pyo.SolverFactory('appsi_highs')
    solver.options['mip_rel_gap'] = mip_gap
    results = solver.solve(model, tee=True)

    gen = value_to_df_2dim(model.GEN, G, T, 'gen')
    commit = value_to_df_2dim(model.COMMIT, G_thermal, T, 'gen')

    curtail = gen_var_cf.merge(gen, on=['r_id', 'hour'], how='inner')
    curtail['curt'] = curtail['cf'] * curtail['existing_cap_mw'] - curtail['gen']

    cost = pyo.value(model.objective)
    status = results.solver.termination_condition

    return gen, commit, curtail, cost, status

In [ ]:
sol_gen_full, sol_commit_full, sol_curtail_full, cost_full, status_full = \
    unit_commitment_full(gen_df_sens, loads_multi, gen_variable_multi, 0.02)

print(f"\nStatus: {status_full}")
print(f"Total cost: ${cost_full:,.2f}")

In [ ]:
sol_gen_full_plot = plot_generation(
    sol_gen_full, sol_curtail_full, gen_df, gen_variable_multi, T_period,
    title='Full UC - Generation Dispatch (Spring Day, 3500 MW Solar)'
)

In [ ]:
plot_commitment(
    sol_commit_full, gen_df, T_period,
    title='Full UC - Commitment Status (Spring Day, 3500 MW Solar)'
)

---

## Exercise: Impact of Natural Gas Price on Unit Commitment — ANSWER KEY

### Step 1: Create a modified generator dataframe with doubled gas price

In [ ]:
# ANSWER: Double the fuel_cost for all generators that use natural gas
gen_df_highgas = gen_df_sens.copy()
gen_df_highgas.loc[gen_df_highgas['fuel_cost'] > 0, 'fuel_cost'] = 5.14

print(f"Original gas price: ${gen_df_sens.loc[gen_df_sens['fuel_cost'] > 0, 'fuel_cost'].iloc[0]:.2f}/MMBtu")
print(f"New gas price: ${gen_df_highgas.loc[gen_df_highgas['fuel_cost'] > 0, 'fuel_cost'].iloc[0]:.2f}/MMBtu")

### Step 2: Solve the Full UC with the high gas price scenario

In [ ]:
# ANSWER: Call unit_commitment_full with gen_df_highgas
sol_gen_highgas, sol_commit_highgas, sol_curtail_highgas, cost_highgas, status_highgas = \
    unit_commitment_full(gen_df_highgas, loads_multi, gen_variable_multi, 0.02)

print(f"\nStatus: {status_highgas}")
print(f"High gas cost: ${cost_highgas:,.2f}")
print(f"Base case cost: ${cost_full:,.2f}")
print(f"Cost increase: ${cost_highgas - cost_full:,.2f} ({(cost_highgas - cost_full) / cost_full * 100:.1f}%)")

**Expected output:**
- Base case cost: ~$903,638
- High gas cost: ~$1,547,589
- Cost increase: ~$643,951 (+71.3%)

### Step 3: Plot and compare results

In [ ]:
# ANSWER: Plot the generation dispatch for the high gas price case
sol_gen_highgas_plot = plot_generation(
    sol_gen_highgas, sol_curtail_highgas, gen_df, gen_variable_multi, T_period,
    title='Full UC (High Gas Price) - Generation Dispatch'
)

In [ ]:
# ANSWER: Plot the commitment status for the high gas price case
plot_commitment(
    sol_commit_highgas, gen_df, T_period,
    title='Full UC (High Gas Price) - Commitment Status'
)

### Step 4: Compare curtailment between base case and high gas price

In [ ]:
# ANSWER: Calculate total curtailment for both scenarios
curtail_base = sol_curtail_full['curt'].sum()
curtail_highgas = sol_curtail_highgas['curt'].sum()
print(f"Total curtailment (base):     {curtail_base:,.0f} MWh")
print(f"Total curtailment (high gas): {curtail_highgas:,.0f} MWh")
print(f"Change: {curtail_highgas - curtail_base:,.0f} MWh")

**Expected output:**
- Base case: ~1,483 MWh curtailed
- High gas: ~836 MWh curtailed
- Change: ~-647 MWh (less curtailment)

---

## Answers to Questions

### Q1: How does the total system cost change? Is it exactly double? Why or why not?

- Base case: **$903,638**
- High gas: **$1,547,589**
- Increase: **+$643,951 (+71.3%)**
- Ratio: **1.71x** — NOT double.

**Why not 2x?** Because not all cost components depend on gas price:
- Renewable generation (solar, wind, hydro) has **zero fuel cost** — their var O&M contribution is unchanged.
- On this high-solar spring day, renewables supply a large share of total energy, so the gas-dependent portion of total cost is well below 100%.
- The optimizer also **adapts** — it decommits thermal units and uses more solar, partially offsetting the price increase.

### Q2: Does the commitment pattern change — are fewer or more thermal units committed?

**Fewer** thermal units are committed:
- CCGTs: 128 → 125 unit-hours (-3)
- CTs: 99 → 67 unit-hours (**-32**)

With higher gas price, every MWh from a committed thermal unit is more expensive. The optimizer decommits CTs wherever possible to avoid burning expensive gas at Pmin. The system "tries harder" to rely on renewables and minimize thermal operation.

### Q3: Does curtailment increase or decrease? What does this tell you?

Curtailment **decreases** from 1,483 MWh to 836 MWh (-647 MWh).

With fewer thermal units committed, fewer plants are stuck at Pmin during midday solar hours. This frees up room for solar generation, so less renewable energy is wasted.

**Key insight:** Higher gas prices actually **improve renewable integration**. Expensive gas gives the optimizer stronger incentive to shut down thermal plants and let solar run. Cheap gas does the opposite — it makes thermal plants "easy to keep on," which crowds out renewables.

### Q4: Which generator type (CCGTs vs CTs) is affected more by the price increase, and why?

**CTs are affected far more** (-32 unit-hours vs -3 for CCGTs). Two reasons:

1. **Higher heat rates:** CTs have heat rates of ~10-12 MMBtu/MWh vs ~7.5 for CCGTs. Doubling gas price hits their marginal cost harder in absolute $/MWh terms.

2. **Flexibility to decommit:** CTs have short min up/down times (1 hour), so the optimizer *can* decommit them flexibly. CCGTs are locked in for 6 hours — even if expensive, they can't easily be toggled.

### Q5: Policy question — what market interventions might a regulator consider?

- **Price caps:** Short-term consumer relief, but can distort investment signals and cause generators to withhold supply if caps fall below marginal cost. Risk of supply shortfalls.

- **Capacity payments:** Pay generators to be available regardless of fuel price — ensures reliability but adds cost to consumers even in normal price periods.

- **Long-term renewable contracts (PPAs):** Fixed-price contracts for solar/wind act as a natural **hedge** — their cost doesn't change when gas prices spike. This exercise demonstrates that renewables reduce cost exposure to fuel volatility.

- **Gas storage requirements:** Mandate utilities to hold gas reserves, smoothing price spikes.

- **Demand response programs:** Pay large consumers to reduce load during price spikes, reducing the need for expensive thermal generation.